this is just a file where i did some tests ... probably only useful for me ..

### First of all - we need to parse the headers and extract declarations for functions and data types

In [1]:
import unittest
import cffi
import importlib
import pycparser
import subprocess

In [2]:
import subprocess

def preprocess(source):
    return subprocess.call(['gcc', '-E', '-P', '-'],
            input=source, stdout=subprocess.PIPE,
            universal_newlines=True, check=True).stdout

In [3]:
def preprocess(source):
    return subprocess.run(['gcc', '-E', '-P', '-'],
                          input=source, stdout=subprocess.PIPE,
                          universal_newlines=True, check=True).stdout

In [4]:
ls *.h

safe_app.h  safe_authenticator.h  safe_core.h


In [5]:
filename='safe_authenticator'

In [6]:
with open(filename + '.h') as f:
    fileContent=f.read()

In [7]:
print(preprocess(fileContent))

typedef signed char int8_t;
typedef short int int16_t;
typedef int int32_t;
typedef long int int64_t;
typedef unsigned char uint8_t;
typedef unsigned short int uint16_t;
typedef unsigned int uint32_t;
typedef unsigned long int uint64_t;
typedef signed char int_least8_t;
typedef short int int_least16_t;
typedef int int_least32_t;
typedef long int int_least64_t;
typedef unsigned char uint_least8_t;
typedef unsigned short int uint_least16_t;
typedef unsigned int uint_least32_t;
typedef unsigned long int uint_least64_t;
typedef signed char int_fast8_t;
typedef long int int_fast16_t;
typedef long int int_fast32_t;
typedef long int int_fast64_t;
typedef unsigned char uint_fast8_t;
typedef unsigned long int uint_fast16_t;
typedef unsigned long int uint_fast32_t;
typedef unsigned long int uint_fast64_t;
typedef long int intptr_t;
typedef unsigned long int uintptr_t;
typedef long int intmax_t;
typedef unsigned long int uintmax_t;
typedef struct RegisteredApp {
 FfiAppExchangeInfo app_info;
 Ffi

In [1]:
import re

In [30]:
from cffi import FFI
ffi = FFI()

In [ ]:
datatypes='''
typedef int int32_t;
typedef struct {
    int32_t error_code;
    char* description;
} FfiResult;
typedef struct {
    void* core_tx;
    void* _core_joiner;
} Authenticator;
'''

In [34]:
ffi.cdef(datatypes)

In [35]:
myFunctions='''
void login(char const* account_locator, char const* account_password, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult const* result, Authenticator* authenticaor));
void create_acc(char const* account_locator, char const* account_password, char const* invitation, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult const* result, Authenticator* authenticator));
'''.replace(' const*','*')

In [37]:
firstFun=myFunctions.splitlines()[1]

In [39]:
fun=re.compile('void \(\*.+?\)\(.+?\)')

In [43]:
callbacks=fun.findall(firstFun)

In [44]:
oneCallback=callbacks[0]

In [46]:
funDef=re.compile('void \(\*.+?\)\(')

In [76]:
arguments=re.compile('\)\(.+\)')

In [64]:
oneCallback

'void (*o_disconnect_notifier_cb)(void* user_data)'

In [65]:
funDef.sub('void(*)(',oneCallback)

'void(*)(void* user_data)'

In [66]:
preparedFun=funDef.sub('void(*)(',callbacks[1])

In [74]:
preparedFun

'void(*)(void* user_data, FfiResult* result, Authenticator* authenticaor)'

In [86]:
def replaceArguments(oneCallback):
    funDef=re.compile('void \(\*.+?\)\(')
    arguments=re.compile('\)\(.+\)')
    oneCallback=funDef.sub('void(*)(',oneCallback)
    allArgs=arguments.findall(oneCallback)[0][2:-1].split(',')
    argString=''
    for i in allArgs:
        argString+=' '.join(i.split()[:-1])+','
    return arguments.sub(')(' + argString[:-1] + ')',oneCallback)
    

In [88]:
callbacks[1]

'void (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticaor)'

In [110]:
def getLocalFunctionDefinition(oneFun):
    funName=re.findall('\(\*.+?\)',oneFun)[0][2:-1]
    allParameters=re.findall('\(.+?\)',oneFun)[-1][1:-1].split(',')
    parameterNames=''
    stringToPrint='void('
    for onePar in allParameters:
        stringToPrint+=' '.join(onePar.split()[:-1])+','
        parameterNames+=onePar.split()[-1]+','
    stringToPrint=stringToPrint[:-1]+')'
    decorator = '@ffi.callback("' + stringToPrint + '")'
    localFunctionName = 'def ' + funName + '(' + parameterNames[:-1] + '):'
    return decorator, localFunctionName

In [101]:
getName=re.compile('void .+?\(')

In [104]:
getName.findall(oneFunction)[0][5:-1]

'create_acc'

In [105]:
getParams=re.compile('\(.+\)')

In [106]:
getParams.findall('def o_cb(user_data,result,authenticaor):')[0][1:-1]

'user_data,result,authenticaor'

In [131]:
classInit='''class Auth:

    def __init__(self,
            name,
            version,
            vendor,
            id,
            addr='http://localhost'):
        self.name = name
        self.version = version
        self.vendor = vendor
        self.id = id
        self.url = addr'''

In [132]:
myFunctions='''
void login(char const* account_locator, char const* account_password, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult const* result, Authenticator* authenticaor));
void create_acc(char const* account_locator, char const* account_password, char const* invitation, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult const* result, Authenticator* authenticator));
'''.replace(' const*','*')
res=[]
print(classInit)
print()
print()
for oneFunction in myFunctions.splitlines()[1:]:
    funName = getName.findall(oneFunction)[0][5:-1]
    callBackFunctions=[]
    for oneCallback in fun.findall(oneFunction):
        oneFunction = oneFunction.replace(oneCallback,replaceArguments(oneCallback))
        callBackFunctions.append(getLocalFunctionDefinition(oneCallback))
        #params = getParams.findall(localFunctionName)[0][1:-1]
    stripCallBacks=re.compile(', void\(\*\).+\)')
    
    params = getParams.findall(stripCallBacks.sub(')',oneFunction))[0][1:-1]
    nakedParams = []
    for oneparam in params.split(','):
        nakedParams.append(oneparam.split()[-1])
    nakedParams = ','.join(nakedParams)
    #print(params)
    print('def ' + funName + '(' + nakedParams + '):')
    print("    ''' " + params + " ''' ")
    for decorator,localFunctionName in callBackFunctions:
        print('    ')
        print('    '+decorator)
        print('    '+localFunctionName)
        print('        pass')
        print('    ')
        
    print('Auth.' + funName + ' = ' + funName)
    print('del(' + funName + ')')
    print()
    print()
    res.append(oneFunction)

class Auth:

    def __init__(self,
            name,
            version,
            vendor,
            id,
            addr='http://localhost'):
        self.name = name
        self.version = version
        self.vendor = vendor
        self.id = id
        self.url = addr


def login(account_locator,account_password,user_data):
    ''' char* account_locator, char* account_password, void* user_data ''' 
    
    @ffi.callback("void(void*)")
    def o_disconnect_notifier_cb(user_data):
        pass
    
    
    @ffi.callback("void(void*,FfiResult*,Authenticator*)")
    def o_cb(user_data,result,authenticaor):
        pass
    
Auth.login = login
del(login)


def create_acc(account_locator,account_password,invitation,user_data):
    ''' char* account_locator, char* account_password, char* invitation, void* user_data ''' 
    
    @ffi.callback("void(void*)")
    def o_disconnect_notifier_cb(user_data):
        pass
    
    
    @ffi.callback("void(void*,FfiResult*,Authenticator*)"

### this is the Authenticator-Class containing all important methods that an authenticator needs to work

In [ ]:
class Auth:

    def __init__(self,
            name,
            version,
            vendor,
            addr='http://localhost'):
        self.name = name
        self.version = version
        self.vendor = vendor
        self.url = addr

#### the login function needs to know at least the two first arguments to work

In [ ]:
def login(account_locator,account_password,user_data):
    ''' char* account_locator, char* account_password, void* user_data ''' 
    
    @ffi.callback("void(void*)")
    def o_disconnect_notifier_cb(user_data):
        pass
    
    
    @ffi.callback("void(void*,FfiResult*,Authenticator*)")
    def o_cb(user_data,result,authenticaor):
        pass
    
Auth.login = login
del(login)

parameters for account creation

account_locator | account_password | invitation | user_data
--- | --- | --- | ---
important; your first login string | important; your second login string | random string in mock routing | no clue


In [ ]:
def create_acc(account_locator,account_password,invitation,user_data):
    ''' char* account_locator, char* account_password, char* invitation, void* user_data ''' 
    
    @ffi.callback("void(void*)")
    def o_disconnect_notifier_cb(user_data):
        pass
    
    
    @ffi.callback("void(void*,FfiResult*,Authenticator*)")
    def o_cb(user_data,result,authenticator):
        pass
    
Auth.create_acc = create_acc
del(create_acc)

In [97]:
for onefunction in res:
    ffi.cdef(onefunction)

In [101]:
getName=re.compile('void .+?\(')

In [104]:
getName.findall(oneFunction)[0][5:-1]

'create_acc'

In [105]:
getParams=re.compile('\(.+\)')

In [106]:
getParams.findall('def o_cb(user_data,result,authenticaor):')[0][1:-1]

'user_data,result,authenticaor'

In [100]:
oneFunction

'void create_acc(char* account_locator, char* account_password, char* invitation, void* user_data, void(*)(void*), void(*)(void*,FfiResult*,Authenticator*));'

In [99]:
res

['',
 'void login(char* account_locator, char* account_password, void* user_data, void(*)(void*), void(*)(void*,FfiResult*,Authenticator*));',
 'void create_acc(char* account_locator, char* account_password, char* invitation, void* user_data, void(*)(void*), void(*)(void*,FfiResult*,Authenticator*));']

In [87]:
replaceArguments(callbacks[1])

'void(*)(void*,FfiResult*,Authenticator*)'

In [85]:
replaceArguments(callbacks[0])

'void(*)(void*)'

In [79]:
arguments.findall(preparedFun)[0][2:-1].split(',')

['void* user_data', ' FfiResult* result', ' Authenticator* authenticaor']

In [50]:
def getLocalFunctionDefinition(oneFun):
    funName=re.findall('\(\*.+?\)',oneFun)[0][2:-1]
    allParameters=re.findall('\(.+?\)',oneFun)[-1][1:-1].split(',')
    parameterNames=''
    stringToPrint='void('
    for onePar in allParameters:
        stringToPrint+=' '.join(onePar.split()[:-1])+','
        parameterNames+=onePar.split()[-1]+','
    stringToPrint=stringToPrint[:-1]+')'
    decorator = '@ffi.callback("' + stringToPrint + '")'
    localFunctionName = 'def ' + funName + '(' + parameterNames[:-1] + '):'
    return decorator, localFunctionName

@ffi.callback("void(void*)")
def o_disconnect_notifier_cb(user_data):


@ffi.callback("void(void*,FfiResult*,Authenticator*)")
def o_cb(user_data,result,authenticaor):




In [4]:
ffi.cdef('''
void login(char* account_locator, char* account_password, void* user_data, void(*)(void*), void(*)(void*, FfiResult*, Authenticator*));
''')

In [5]:
ffi.cdef('''
void create_acc(char* account_locator, char* account_password, char* invitation, void* user_data, void(*)(void*), void(*)(void*, FfiResult*, Authenticator*));
''')

In [6]:
def getCString(cString, length):
    completeString=b''
    for i in range(length):
        completeString+=cString[i]
    return completeString

login

In [19]:
@ffi.callback("void(void*)")
def o_disconnect_notifier_cb(user_data):
    print('o_disconnect_notifier_cb')

@ffi.callback("void(void*,FfiResult*,Authenticator*)")
def o_cb(user_data,result,authenticaor):
    try:
        print(getCString(result.description,77))
    except:
        pass
    print(result.error_code)
    print('o_cb')

create account

In [18]:
@ffi.callback("void(void*)")
def o_disconnect_notifier_cb_create_acc(user_data):
    print('hallö')

@ffi.callback("void(void*,FfiResult*,Authenticator*)")
def o_cb_create_acc(user_data,result,authenticator):
    try:
        print(getCString(result.description,77))
    except:
        pass
    print(result.error_code)


In [10]:
pwd

'/home/riddim/safe/pySafe/bindingsGeneration'

In [8]:
lib = ffi.dlopen('../../safe_client_libs/target/debug/libsafe_authenticator.so')

In [13]:
account_locator=ffi.new('char[]',b'superkalifragilisti9')
password=ffi.new('char[]',b'1l0v3myl1v3')

invitation=ffi.new('void[72]',b'1l0v3myl1v3')

In [10]:
invitation=ffi.new('char[]',b'testData')

In [14]:
userData=ffi.new('char[]',b'')

In [34]:
lib.login(account_locator,password,userData,o_disconnect_notifier_cb,o_cb)

o_cb


In [39]:
lib.login(account_locator,password,userData,o_disconnect_notifier_cb,o_cb)

b'Core error: Unexpected: Could not connect to the SAFE Network\x00\x00\x00P\x00\x00\x00\x00\x00\x00\x00\x91\x00\x00\x00\x00'
-2000
o_cb


In [14]:
lib.login(account_locator,password,userData,o_disconnect_notifier_cb,o_cb)

b'Core error: Routing client error -> Account does not exist for client\x00\x00\x00\xd1\x02\x00\x00\x00'
-101
o_cb


In [16]:
lib.create_acc(account_locator,password,invitation,userData,o_disconnect_notifier_cb_create_acc,o_cb_create_acc)

b'Core error: Routing client error -> Account already exists for client\x00\x00\x00\x91\x00\x00\x00\x00'
-102


In [17]:
lib.login(account_locator,password,userData,o_disconnect_notifier_cb,o_cb)

0
o_cb


In [25]:
account_locator2=ffi.new('char[]',b'mySuperSafeTest2Acc0unt')
password2=ffi.new('char[]',b'mySuperSafeTes2tAcc0unt')

In [26]:
invitation2=ffi.new('char[]',b'testData')

In [27]:
userData2=ffi.new('char[]',b'')

In [28]:
lib.create_acc(account_locator2,password2,invitation2,userData2,o_disconnect_notifier_cb_create_acc,o_cb_create_acc)

0


In [29]:
lib.login(account_locator2,password2,userData2,o_disconnect_notifier_cb,o_cb)

0
o_cb


In [ ]:
lib.login('hallo')

In [6]:
includes = preprocess(fileContent)

In [11]:
typedefs=''
functionDeclarations=''
for i in includes.splitlines():
    if i[:4] == 'void':
        functionDeclarations+='\n'+i
    else:
        typedefs+='\n'+i

In [ ]:
void app_unregistered(
    uint8_t const* bootstrap_config,
    uintptr_t bootstrap_config_len, 
    void* user_data, 
        void (*o_disconnect_notifier_cb)(void* user_data)
        void (*o_cb)(void* user_data, FfiResult const* result, App* app));

In [16]:
import re

In [13]:
from cffi import FFI
ffi = FFI()

In [14]:
lib = ffi.dlopen('../target/debug/libsafe_app.so')

OSError: cannot load library ../target/debug/libsafe_app.so: ../target/debug/libsafe_app.so: cannot open shared object file: No such file or directory.  Additionally, ctypes.util.find_library() did not manage to locate a library called '../target/debug/libsafe_app.so'

In [15]:
ffi.cdef('''
typedef signed char int8_t;
''')

In [4]:
@ffi.callback("void(int8_t)")
def myPyFunction(myVal):
    print('hallo')
    print(myVal)

In [210]:
declLine='void mdata_get_version(App const* app, FfiMDataInfo const* info, void* user_data, void (*o_cb)(void* user_data, FfiResult const* result, uint64_t version));'

In [211]:
declLine2='void app_registered(char const* app_id, FfiAuthGranted const* auth_granted, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult const* result, App* app));'

In [8]:
declLine2='void login(char* account_locator, char* account_password, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticaor));'

In [15]:
declLine2='void create_acc(char const* account_locator, char const* account_password, char const* invitation, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult const* result, Authenticator* authenticator));'

In [16]:
fun=re.compile('void \(\*.+?\)\(.+?\)')

In [18]:
fun.findall(declLine)

NameError: name 'declLine' is not defined

In [19]:
fun.findall(declLine2)

['void (*o_disconnect_notifier_cb)(void* user_data)',
 'void (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticaor)']

In [17]:
fun.findall(declLine2)

['void (*o_disconnect_notifier_cb)(void* user_data)',
 'void (*o_cb)(void* user_data, FfiResult const* result, Authenticator* authenticator)']

In [18]:
for oneFun in fun.findall(declLine2):
    funName=re.findall('\(\*.+?\)',oneFun)[0][2:-1]
    allParameters=re.findall('\(.+?\)',oneFun)[-1][1:-1].split(',')
    parameterNames=''
    stringToPrint='void('
    for onePar in allParameters:
        stringToPrint+=' '.join(onePar.split()[:-1])+','
        parameterNames+=onePar.split()[-1]+','
    stringToPrint=stringToPrint[:-1]+')'
    print('@ffi.callback("' + stringToPrint + '")')
    print('def ' + funName + '(' + parameterNames[:-1] + '):')
    print()
    print()

@ffi.callback("void(void*)")
def o_disconnect_notifier_cb(user_data):


@ffi.callback("void(void*,FfiResult const*,Authenticator*)")
def o_cb(user_data,result,authenticator):




In [237]:
dataTypesExcample='''typedef int int32_t;
    typedef unsigned long int uint_fast32_t;
    typedef unsigned long int uint_fast64_t;
typedef struct {
    int32_t error_code;
    char* description;
} FfiResult;
typedef struct {
    int32_t error_code;
    FfiResult* nonono;
} testStruct;
typedef struct {
    unsigned long int error_code;
    unsigned long int* error_code2;
    testStruct* nonono;
} testStruct2;'''

In [238]:
oneliners=re.compile('typedef.+?;')

In [239]:
myStructTypes=re.compile('typedef struct .+?}.+?;', flags=re.DOTALL)

In [240]:
allOneLiners=oneliners.findall(dataTypesExcample)
allOneLiners

['typedef int int32_t;',
 'typedef unsigned long int uint_fast32_t;',
 'typedef unsigned long int uint_fast64_t;']

In [241]:
allMyStructs=myStructTypes.findall(dataTypesExcample)
for oneStruct in allMyStructs:
    print(oneStruct)

typedef struct {
    int32_t error_code;
    char* description;
} FfiResult;
typedef struct {
    int32_t error_code;
    FfiResult* nonono;
} testStruct;
typedef struct {
    unsigned long int error_code;
    unsigned long int* error_code2;
    testStruct* nonono;
} testStruct2;


---

In [253]:
dictWithTypes={}

In [254]:
for oneStruct in allMyStructs:
    allElements=[]
    for oneline in oneStruct.splitlines()[1:-1]:
        allElements.append(' '.join(oneline.split()[:-1]))
    myKey=oneStruct.splitlines()[-1].split()[-1][:-1]
    dictWithTypes[myKey]=allElements

In [255]:
dictWithTypes

{'FfiResult': ['int32_t', 'char*'],
 'testStruct': ['int32_t', 'FfiResult*'],
 'testStruct2': ['unsigned long int', 'unsigned long int*', 'testStruct*']}

In [281]:
def printDeclaration(varName,varType):
    neededVars=dictWithTypes[varType.strip('*')]
    #print(neededVars)
    containedVars=[]
    for i,allVarstypes in enumerate(neededVars):
        if allVarstypes.strip('*') in dictWithTypes.keys():
            printDeclaration(varName+'_'+str(i),allVarstypes)
            containedVars.append(varName+'_'+str(i))
        else:
            stringToPrint=varName + '_' + str(i) + ' =  ffi.new(\'' + allVarstypes + '\',0)'
            containedVars.append(varName+'_'+str(i))
            print(stringToPrint)
    stringToPrint=varName + ' = ffi.new(\'' + varType + '\','
    if len(containedVars) > 1:
        stringToPrint+='['
        for one in containedVars:
            stringToPrint+=one+','
        stringToPrint=stringToPrint[:-1]+']'
    else:
        stringToPrint+=containedVars[0]
    stringToPrint+=')'
    print(stringToPrint)

In [282]:
printDeclaration('test','testStruct2')

test_0 =  ffi.new('unsigned long int',0)
test_1 =  ffi.new('unsigned long int*',0)
test_2_0 =  ffi.new('int32_t',0)
test_2_1_0 =  ffi.new('int32_t',0)
test_2_1_1 =  ffi.new('char*',0)
test_2_1 = ffi.new('FfiResult*',[test_2_1_0,test_2_1_1])
test_2 = ffi.new('testStruct*',[test_2_0,test_2_1])
test = ffi.new('testStruct2',[test_0,test_1,test_2])


In [74]:
class SafePy:        
    def __init__(self, myName, myParameter):
        self.name = myName
        self.parameter1 = [myParameter]
        
        
    def getNewVal(self,myVal):
        @ffi.callback("void(int8_t)")
        def myPyFunction(myVal):
            self.parameter1.append(myVal)
        
        lib.functionCallBack(myVal,myPyFunction)

In [75]:
newSafe=SafePy('rid',17)

In [76]:
secondSafe=SafePy('dim',10)

In [77]:
secondSafe.getNewVal(11)

In [78]:
secondSafe.parameter1

[10, 33, 22]

In [79]:
newSafe.parameter1

[17]

In [80]:
newSafe.getNewVal(3)

In [81]:
newSafe.parameter1

[17, 9, 6]

In [82]:
newSafe.getNewVal(14)

In [83]:
newSafe.parameter1

[17, 9, 6, 42, 28]

In [84]:
secondSafe.parameter1

[10, 33, 22]

In [1]:
from cffi import FFI
ffi = FFI()

In [2]:
lib = ffi.dlopen('../target/debug/libsafe_app.so')

In [3]:
ffi.cdef('''
typedef signed char int8_t;
typedef int int32_t;
typedef struct {
    int32_t error_code;
    char* description;
} FfiResult;
''')

In [4]:
def getCString(cString, length):
    completeString=b''
    for i in range(length):
        completeString+=cString[i]
    return completeString

In [8]:
class FfiRes:
    def __init__(self, myStr, num):
        self.innerString = ffi.new('char[]',myStr)
        self.stringLength=len(myStr)
        self.entity = ffi.new('FfiResult*',[num,self.innerString])
        
    def getNum(self):
        return self.entity.error_code
    
    def getString(self):
        return getCString(self.entity.description,self.stringLength)

In [9]:
newF=FfiRes(b'waaaaaahahaaaaa!',2)

In [10]:
newF.entity

<cdata 'FfiResult *' owning 16 bytes>

In [5]:
@ffi.callback("void(FfiResult *)")
def myPyFunction(myFfiRes):
    def getCString(cString, length):
        completeString=b''
        for i in range(length):
            completeString+=cString[i]
        return completeString
    print(myFfiRes.error_code)
    print(getCString(myFfiRes.description,35))

In [6]:
myPyFunction

<cdata 'void(*)(FfiResult *)' calling <function myPyFunction at 0x7f3ca15b5b70>>

In [7]:
ffi.cdef('''
void functionCallBack2(int32_t number, FfiResult* firstRes, void(*)(FfiResult *));
''')

In [11]:
lib.functionCallBack2(17,newF.entity,myPyFunction)

17
b'now this is getting fun!\x00\x00\x00\x00\x00\x00\x00\x000\x00\x00'


In [1]:
from cffi import FFI
ffi = FFI()

In [2]:
ffi.cdef('''
typedef signed char int8_t;
typedef struct {
    int8_t* sym_enc_key;
    int8_t sym_enc_key2;
} Registered;
void pyTestFun(int8_t* number, Registered* myStruct );
''')

In [3]:
lib = ffi.dlopen('../target/debug/libsafe_app.so')

In [4]:
testVar1=ffi.new('int8_t*',17)

In [6]:
randomNumber=ffi.new('int8_t*',7)

In [8]:
randomNumber[0]

7

creating the struct with containing (pointer to) numbers as defined above

In [9]:
myStruct=ffi.new('Registered *',[randomNumber,2])

In [10]:
lib.pyTestFun(testVar1,myStruct)

In [24]:
res1.description[2]

b'u'

In [25]:
res2.description[2]

b'u'

In [11]:
testVar1[0]

20

In [12]:
myStruct.sym_enc_key[0]

8

In [14]:
myStruct.sym_enc_key2

4

In [15]:
lib.pyTestFun(testVar1,myStruct)

In [16]:
testVar1[0]

23

In [17]:
myStruct.sym_enc_key[0]

9

In [18]:
myStruct.sym_enc_key2

8

### playing with strings

In [1]:
def getCString(cString, length):
    completeString=b''
    for i in range(length):
        completeString+=cString[i]
    return completeString

In [2]:
from cffi import FFI
ffi = FFI()

In [3]:
ffi.cdef('''
typedef signed char int8_t;
typedef int int32_t;
typedef struct {
    int32_t error_code;
    char* description;
} FfiResult;
typedef struct {
    int8_t* sym_enc_key;
    int8_t sym_enc_key2;
} Registered;
void pyTestFun(int8_t* number, Registered* myStruct );
void pyTestFun2(FfiResult* firstRes, FfiResult* secondRes );
void pyTestFun3(FfiResult* firstRes, FfiResult* secondRes );
''')

In [4]:
lib = ffi.dlopen('../target/debug/libsafe_app.so')

In [41]:
class FfiRes:
    def __init__(self, myStr, num):
        self.innerString = ffi.new('char[]',myStr)
        self.stringLength=len(myStr)
        self.entity = ffi.new('FfiResult*',[num,self.innerString])
        
    def getNum(self):
        return self.entity.error_code
    
    def getString(self):
        return getCString(self.entity.description,self.stringLength)

In [47]:
newF=FfiRes(b'ha! - das wird richtig gut',2)

In [46]:
newF2=FfiRes(b'hasdfh gut',3)

In [48]:
lib.pyTestFun3(newF.entity,newF2.entity)

In [43]:
newF.getNum()

2

In [51]:
newF2.stringLength = 35

In [52]:
newF2.getString()

b'hello - now this is getting fun!\x00\x0f\xcf'

In [5]:
myString=ffi.new('char[]',b'lalilala')

In [6]:
myString2=ffi.new('char[]',b'wer\'re getting closer')

In [7]:
res1=ffi.new('FfiResult*',[1,myString])

In [8]:
res2=ffi.new('FfiResult*',[2,myString2])

---

In [9]:
getCString(res1.description,23)

b'lalilala\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [10]:
res1.error_code

1

In [11]:
getCString(res2.description,23)

b"wer're getting closer\x00\x00"

---

In [12]:
lib.pyTestFun2(res1,res2)

In [13]:
res1.error_code

17

---

In [14]:
getCString(res1.description,23)

b"wer're getting closer\x00\x00"

In [15]:
getCString(res2.description,23)

b"wer're getting closer\x00\x00"

In [16]:
res2.description[0]=b'H'

In [17]:
getCString(res1.description,23)

b"Her're getting closer\x00\x00"

In [18]:
getCString(res2.description,23)

b"Her're getting closer\x00\x00"

In [12]:
lib.pyTestFun3(res1,res2)

In [13]:
res1.error_code

27

In [14]:
getCString(res1.description,23)

b'lalilala\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [15]:
getCString(res2.description,35)

b'hello - now this is getting fun!\x00\x00\x00'

In [22]:
newFfi=getFfiRes(b'hallo',15)

In [23]:
getCString(newFfi.description,23)

b'@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00(bnME\x7f\x00'

In [21]:
newFfi.error_code

15

##### the following is just as hint for later that it should be possible to do automatic preprocessing to extract most of the info out of the c header files

In [2]:
from cffi import FFI
ffi = FFI()

In [53]:
import unittest
import cffi
import importlib
import pycparser
import subprocess

def preprocess(source):
    return subprocess.call(['gcc', '-E', '-P', '-'],
            input=source, stdout=subprocess.PIPE,
            universal_newlines=True, check=True).stdout

In [4]:
def preprocess(source):
    return subprocess.run(['gcc', '-E', '-P', '-'],
                          input=source, stdout=subprocess.PIPE,
                          universal_newlines=True, check=True).stdout

In [5]:
ls *.h

newtest.h  safe_app.h  safe_authenticator.h  safe_core.h  test.h


In [21]:
filename='safe_app'

In [22]:
with open(filename + '.h') as f:
    fileContent=f.read()

In [23]:
includes = preprocess(fileContent)

In [24]:
print(includes)

typedef signed char int8_t;
typedef short int int16_t;
typedef int int32_t;
typedef long int int64_t;
typedef unsigned char uint8_t;
typedef unsigned short int uint16_t;
typedef unsigned int uint32_t;
typedef unsigned long int uint64_t;
typedef signed char int_least8_t;
typedef short int int_least16_t;
typedef int int_least32_t;
typedef long int int_least64_t;
typedef unsigned char uint_least8_t;
typedef unsigned short int uint_least16_t;
typedef unsigned int uint_least32_t;
typedef unsigned long int uint_least64_t;
typedef signed char int_fast8_t;
typedef long int int_fast16_t;
typedef long int int_fast32_t;
typedef long int int_fast64_t;
typedef unsigned char uint_fast8_t;
typedef unsigned long int uint_fast16_t;
typedef unsigned long int uint_fast32_t;
typedef unsigned long int uint_fast64_t;
typedef long int intptr_t;
typedef unsigned long int uintptr_t;
typedef long int intmax_t;
typedef unsigned long int uintmax_t;
void access_container_refresh_access_info(App const* app, void* u